<a href="https://colab.research.google.com/github/shankch/reverse-image/blob/main/3_benchmark_compare_algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similarity Search: Benchmarking the algorithms

We benchmark the algorithms based on the time it takes to index images and locate the most similar image based on its features using the UC Merced Land Use dataset. We also experiment with t-SNE and PCA.

### Understanding the time it takes to index images and locate the most similar image based on its features

For these experiments we will use the features of the UC Merced Land Use dataset that we read above.

First, let's choose a random image to experiment with. We will be using the same image for all the following experiments. Note: the results may change if the image is changed.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir data
!cp -a drive/MyDrive/c99t1/* data/

Mounted at /content/drive


In [48]:
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [49]:
filenames = pickle.load(open('data/filenames-udset.pickle', 'rb'))
feature_list = pickle.load(open('data/features-udset-resnet.pickle',
                                'rb'))
class_ids = pickle.load(open('data/class_ids-udset.pickle', 'rb'))

In [50]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  2100
Number of features per image =  2048


In [51]:
random_image_index = random.randint(0, num_images)
print (random_image_index)

638


### Standard features

The following experiments are based on the ResNet-50 features derived from the images of the UC Merced Land Use dataset. 

### Standard features + Brute Force Algorithm on one image

We will be timing the indexing for various Nearest Neighbors algorithms, so let's start with timing the indexing for the Brute force algorithm. While running terminal commands in iPython like the `timeit` command, the variables are not stored in memory, so we need to rerun the same command to compute and store the results in the variable. 

In [52]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)

100 loops, best of 5: 3.4 ms per loop


Now, let's look at the time it takes to search for the nearest neighbors for the selected random image using the trained model with the Brute force algorithm.

In [53]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

100 loops, best of 5: 15.7 ms per loop


###  Standard features + k-d Tree Algorithm  on one image

Now let's turn our attention to the next nearest neighbors algorithm, the k-d tree. Let's time the indexing for the k-d tree algorithm.

In [54]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)

1 loop, best of 5: 358 ms per loop


Now, time the search for the same random image using the k-d tree trained model.

In [55]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

100 loops, best of 5: 9.34 ms per loop


###  Standard features + Ball Tree Algorithm  on one image

Finally, its time for our last nearest neighbors algorithm - the Ball Tree algorithm. As before, let's calculate the time it takes to train the model.

In [56]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)

1 loop, best of 5: 249 ms per loop


As before, let's time the search for the Ball Tree model.

In [57]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

100 loops, best of 5: 6.3 ms per loop


We will increase the number of our test images so that we can experiment with how the scalability of different nearest neighbors algorithms change. Let's choose a random set of 100 or 1000 images to experiment. 

Note: the results may change if any of the images are changed

Generate a list of images to do the next set of experiments on.

In [ ]:
random_image_indices = random.sample(range(0, num_images), 1000)
print (len(random_image_indices))
random_feature_list = [ feature_list[each_index] for each_index in random_image_indices ]
print (random_feature_list)

### Standard features + Brute Force Algorithm on a set of images

Time the search for the Brute force algorithm.

In [59]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 5: 406 ms per loop


### Standard features +  k-d Tree Algorithm on a set of images

Time the search for the k-d tree algorithm.

In [60]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 5: 8.69 s per loop


### Standard features +  Ball Tree Algorithm on a set of images

Time the search for the Ball Tree algorithm.

In [61]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 5: 5.64 s per loop


### PCA

Now we have seen the time it takes to index and search using nearest neighbor algorithms on the full feature length. We can use PCA to compress the features and reduce the time. As before we set the number of features intended.

In [62]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions, len(feature_list[0]))

Train the PCA model with the number of desired feature dimensions.

In [63]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()
#print(feature_list_compressed)

Let's try to understand the importance of each of the resultant features. The numbers displayed below show the relative importance of the first 20 features.

In [64]:
print(pca.explained_variance_ratio_[0:20])

[0.0980664  0.05130073 0.04690164 0.03724571 0.03128028 0.02667752
 0.02137872 0.0166554  0.01613193 0.01441177 0.01295428 0.01177393
 0.01119185 0.01053516 0.01029318 0.00927576 0.00880797 0.00872749
 0.00809281 0.00732378]


Repeat the timing experiments. We use the same random image to experiment.
Note: the results may change if the image is changed.

### PCA + Brute Force Algorithm on one image

Let's time the indexing for the brute force algorithm.

In [65]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)

100 loops, best of 5: 9.92 ms per loop


We will now time the search for the brute force algorithm.

In [66]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

The slowest run took 5.35 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 1.05 ms per loop


###  PCA + k-d Tree Algorithm  on one image

Time the indexing for the k-d tree algorithm.

In [67]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)

10 loops, best of 5: 25.6 ms per loop


Time the search for the k-d tree algorithm.

In [68]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

1000 loops, best of 5: 973 µs per loop


###  PCA + Ball Tree Algorithm  on one image

Time the indexing for the ball tree algorithm.

In [69]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)

10 loops, best of 5: 20.1 ms per loop


Time the search for the ball tree algorithm.

In [70]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

The slowest run took 7.91 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 801 µs per loop


We use the same random indices to experiment. Note: the results may change if any of the images are changed.

Generate a list of images to do the next set of experiments on.

In [71]:
random_feature_list_compressed = [
    feature_list_compressed[each_index] for each_index in random_image_indices
]

### PCA + Brute Force Algorithm on a set of images

Time the search for the brute force algorithm.

In [72]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

10 loops, best of 5: 60.8 ms per loop


### PCA + k-d Tree Algorithm on a set of images

Time the search for the k-d tree algorithm.

In [73]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 5: 346 ms per loop


### PCA + Ball Tree Algorithm on a set of images

Time the search for the Ball Tree algorithm.

In [74]:
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 5: 260 ms per loop


### Annoy 

Make sure you have `annoy` installed. You can install it using pip, `pip3 install annoy`.

In [75]:
!pip3 install annoy
from annoy import AnnoyIndex

In [76]:
# Time the indexing for Annoy
t = AnnoyIndex(2048)  # Length of item vector that will be indexed
starttime = time.time()
for i in range(num_images):
    feature = feature_list[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/udset_index.ann')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


0.7478163242340088


True

### Annoy on one image 

Time the search for one image for Annoy.

In [77]:
u = AnnoyIndex(2048)
%timeit u.get_nns_by_vector(feature_list[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list[random_image_index],
                              5,
                              include_distances=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


1000 loops, best of 5: 325 µs per loop


Helper function to time the search for multiple images for Annoy. Perform the search for the same image multiple times to get an average value.


In [78]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(feature_list[random_image_index],
                                      5,
                                      include_distances=True)
        #print(indexes)

### Annoy on a set of images

Time the search for multiple images for Annoy.

In [79]:
%time calculate_annoy_time()

CPU times: user 38.8 ms, sys: 263 µs, total: 39.1 ms
Wall time: 36.8 ms


### PCA + Annoy

Now, let's time the indexing for Annoy for the PCA generated features.

In [80]:
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
    feature = feature_list_compressed[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/udset_index.ann')

0.008975028991699219


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  This is separate from the ipykernel package so we can avoid doing imports until


True

### PCA + Annoy for one image

Time the search for one image for Annoy.

In [81]:
u = AnnoyIndex(num_feature_dimensions)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                              5,
                              include_distances=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


100000 loops, best of 5: 2.73 µs per loop


Helper function to time the search for multiple images for Annoy. Perform the search for the same image multiple times to get an average value.


In [82]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(
            feature_list_compressed[random_image_index],
            5,
            include_distances=True)

### PCA + Annoy on a set of images

Time the search for multiple images for Annoy.

In [83]:
%time calculate_annoy_time()

CPU times: user 1.12 ms, sys: 154 µs, total: 1.28 ms
Wall time: 1.57 ms


### NMS Lib

In [84]:
!pip install nmslib
import nmslib

In [85]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(feature_list_compressed)
index.createIndex({'post': 2}, print_progress=True)

### NMS Lib on one image

In [86]:
# Query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(feature_list_compressed[random_image_index], k=5)
ids, distances = index.knnQuery(feature_list_compressed[random_image_index],
                                k=5)

The slowest run took 11.31 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 22 µs per loop


### NMS Lib on a set of images

In [87]:
# Get all nearest neighbors for all the datapoint
# using a pool of 4 threads to compute
%timeit index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)
neighbors = index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)

10 loops, best of 5: 41 ms per loop


### Falconn


In [88]:
!pip install falconn
import falconn

In [89]:
# Setup different parameters for Falonn
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_feature_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable

# Train the Falconn model
falconn.compute_number_of_hash_functions(16, parameters)

### Falconn on a set of images

In [90]:
dataset = np.array(feature_list_compressed)
a = np.random.randn(8677, 100)
a /= np.linalg.norm(a, axis=1).reshape(-1, 1)
dataset = a

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

searchQuery = np.array(feature_list_compressed[random_image_index])
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

CPU times: user 12.3 ms, sys: 0 ns, total: 12.3 ms
Wall time: 12.8 ms
The slowest run took 27.38 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 4.32 µs per loop


### PCA + Annoy

In [91]:
# Time the indexing for Annoy for the PCA generated features
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
    feature = dataset[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 40 trees
t.save('data/udset_index.ann')

0.039916038513183594


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  after removing the cwd from sys.path.


True

In [92]:
u = AnnoyIndex(num_feature_dimensions)
# Time the search for one image for Annoy
%timeit u.get_nns_by_vector(dataset[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(dataset[random_image_index],
                              5,
                              include_distances=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


10000 loops, best of 5: 15.4 µs per loop


#TBD Record the benchmark data in a table and display to see relative speed 